In [3]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth

scope='user-library-read playlist-modify-public playlist-modify-private'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="6584944ee0d9495480193d9997a9efb7",
                                               client_secret="ac4205f012b64179acd46c0fbdb33f36",
                                               redirect_uri="http://127.0.0.1:5000/redirect",
                                               scope=scope))

songs = []
pl = sp.current_user_playlists()['items'];
id = pl[2]['id']
playlist = sp.playlist_tracks(id)

tracks_data = []

for track in playlist['items']:
    
    track_data = {
            'danceability': None,
            'energy': None,
            'loudness': None,
            'speechiness': None,
            'acousticness': None,
            'instrumentalness': None,
            'liveness': None,
            'valence': None,
            'tempo': None,
            'id': track['track']['id']
        }
    
    audio_features = sp.audio_features([track['track']['uri']])[0]
    if audio_features:
        track_data['danceability'] = audio_features['danceability']
        track_data['energy'] = audio_features['energy']
        track_data['loudness'] = audio_features['loudness']
        track_data['speechiness'] = audio_features['speechiness']
        track_data['acousticness'] = audio_features['acousticness']
        track_data['instrumentalness'] = audio_features['instrumentalness']
        track_data['liveness'] = audio_features['liveness']
        track_data['valence'] = audio_features['valence']
        track_data['tempo'] = audio_features['tempo']
        
    tracks_data.append(track_data)

playlist_features = pd.DataFrame(tracks_data)
playlist_features.drop_duplicates('id')

playlist_features.head()


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.716,0.531,-7.355,0.1220,0.0703,0.000000,0.2240,0.344,71.994,0UtnpKaReKUg2GquaSxCyD
1,0.542,0.466,-8.665,0.0831,0.1410,0.000445,0.1250,0.124,191.385,7rbECVPkY5UODxoOUVKZnA
2,0.796,0.766,-5.974,0.2380,0.0742,0.000000,0.0827,0.558,110.034,3iVcZ5G6tvkXZkZKlMpIUs
3,0.837,0.636,-7.643,0.0860,0.0395,0.001250,0.3420,0.274,145.972,2t8yVaLvJ0RenpXUIAC52d
4,0.664,0.447,-7.272,0.2890,0.1120,0.000000,0.0910,0.659,83.138,5cc9Zbfp9u10sfJeKZ3h16


In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
all_songs_df = pd.read_csv(r'C:\Users\alext\OneDrive\Desktop\ml_data\tracksgenres.csv')

#rearrange columns to match the user dataframe
all_songs_df_order = ['artists','genres', 'id', 'track_pop','danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']
all_songs_features_order = ['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','id']
all_songs_df = all_songs_df[all_songs_df_order]
all_songs_features = all_songs_df[all_songs_features_order] 

all_songs_features.head()


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.700,0.219,-11.896,0.4230,0.992,0.000000,0.178,0.539,106.112,1YZkudiD6LfY5vmZKdOTyO
1,0.439,0.311,-14.167,0.3270,0.960,0.243000,0.211,0.392,77.238,36OG5Ep6IPA4Mj6TxoTpcw
2,0.357,0.185,-13.705,0.0309,0.802,0.000000,0.105,0.303,109.524,0JqKK01MXbnDhBKKdcUCnX
3,0.576,0.299,-10.515,0.0661,0.878,0.000009,0.409,0.983,201.395,2iKCL9YGGTpDPen0LhzcX2
4,0.252,0.234,-10.615,0.0303,0.875,0.000000,0.371,0.295,95.664,2fwms7R3b5N8aBnJ1CaltH


In [5]:
all_songs_df.head()

,artists,genres,id,track_pop,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,['Guru Dutt'],[],1YZkudiD6LfY5vmZKdOTyO,5,0.700,0.219,-11.896,0.4230,0.992,0.000000,0.178,0.539,106.112
1,['Guru Dutt'],[],36OG5Ep6IPA4Mj6TxoTpcw,6,0.439,0.311,-14.167,0.3270,0.960,0.243000,0.211,0.392,77.238
2,['The De Castro Sisters'],[],0JqKK01MXbnDhBKKdcUCnX,15,0.357,0.185,-13.705,0.0309,0.802,0.000000,0.105,0.303,109.524
3,['The De Castro Sisters'],[],2iKCL9YGGTpDPen0LhzcX2,8,0.576,0.299,-10.515,0.0661,0.878,0.000009,0.409,0.983,201.395
4,['The De Castro Sisters'],[],2fwms7R3b5N8aBnJ1CaltH,1,0.252,0.234,-10.615,0.0303,0.875,0.000000,0.371,0.295,95.664


In [6]:
playlist_features.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.716,0.531,-7.355,0.1220,0.0703,0.000000,0.2240,0.344,71.994,0UtnpKaReKUg2GquaSxCyD
1,0.542,0.466,-8.665,0.0831,0.1410,0.000445,0.1250,0.124,191.385,7rbECVPkY5UODxoOUVKZnA
2,0.796,0.766,-5.974,0.2380,0.0742,0.000000,0.0827,0.558,110.034,3iVcZ5G6tvkXZkZKlMpIUs
3,0.837,0.636,-7.643,0.0860,0.0395,0.001250,0.3420,0.274,145.972,2t8yVaLvJ0RenpXUIAC52d
4,0.664,0.447,-7.272,0.2890,0.1120,0.000000,0.0910,0.659,83.138,5cc9Zbfp9u10sfJeKZ3h16


In [7]:
def generate_playlist_feature(complete_feature_set, playlist_df):

    # Find song features in the playlist
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]
    # Find all non-playlist song features
    complete_feature_set_playlist_output = complete_feature_set_playlist.drop(columns = "id")
    return complete_feature_set_playlist_output.sum(axis = 0)

In [8]:
playlistfeatures = generate_playlist_feature(playlist_features, all_songs_df)

i = 0
while i <= len(playlist_features):
    playlistfeatures[i] = playlistfeatures[i]/len(playlist_features)
    i+=1


# "single song" matrix
print(playlistfeatures)

danceability          0.435000
energy                0.373600
loudness             -4.456400
speechiness           0.081420
acousticness          0.050940
instrumentalness      0.000339
liveness              0.549700
valence               0.956000
tempo               447.391000
dtype: float64


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

def generate_playlist_recos(df, features, nonplaylist_features):

    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    
    # Find cosine similarity between the playlist and the complete song set
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    
    return non_playlist_df_top_40

In [10]:
recos = generate_playlist_recos(all_songs_df, playlistfeatures, all_songs_features)

recos.head(20)



,artists,genres,id,track_pop,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,sim
449596,['FTISLAND'],"['k-pop', 'k-pop boy group', 'k-rock', 'korean...",2DGKTUZ3ywykT9hcqiYgR6,38,0.275,0.791,-2.215,0.0546,0.289000,0.000000,0.1480,0.319,205.416,0.999994
355225,['back number'],"['j-pop', 'j-rock']",3I51KkjhqwuXLcslSz0Q02,52,0.394,0.797,-2.337,0.0571,0.125000,0.000000,0.1060,0.655,204.426,0.999993
42955,['WANIMA'],"['j-pop', 'j-poprock', 'j-punk', 'j-rock', 'ja...",6UMCZw9hFiK5aBm7pQxgkO,44,0.265,0.926,-1.900,0.0946,0.006950,0.000000,0.3390,0.520,203.971,0.999993
120880,['ELLEGARDEN'],"['j-pop', 'j-poprock', 'j-rock', 'japanese alt...",35sboDeBvRDcK8GEAlaBWG,38,0.256,0.754,-1.759,0.0634,0.120000,0.000021,0.1410,0.595,171.946,0.999992
424113,['Aimyon'],"['j-acoustic', 'j-pop']",20mJp0Ix72iR2EF8BdCfE5,51,0.473,0.860,-2.022,0.0921,0.056600,0.000004,0.1400,0.728,202.937,0.999992
424105,['Aimyon'],"['j-acoustic', 'j-pop']",34s49nvRfQy1kzx7DBqVRC,45,0.474,0.860,-2.022,0.0921,0.056600,0.000004,0.1400,0.729,202.937,0.999992
87402,['Tři Sestry'],"['czech folk', 'czech punk', 'czech rock']",0neX4Gme715RYqusNqKWem,19,0.277,0.932,-2.105,0.0592,0.000270,0.001160,0.0507,0.475,200.343,0.999992
150421,['Kaizers Orchestra'],"['dark cabaret', 'norwegian alternative rock',...",52LcpzEayzgty7oYWixLtJ,50,0.244,0.653,-2.268,0.0331,0.072400,0.000000,0.0715,0.580,181.437,0.999992
123415,['Yuzu'],"['j-acoustic', 'j-pop', 'j-poprock', 'j-rock']",4TtFo8HtLCyfUSCPG4YulX,27,0.416,0.863,-2.223,0.0982,0.079500,0.000000,0.1810,0.866,207.580,0.999991
87739,['Depresszió'],"['hungarian metal', 'hungarian punk', 'hungari...",149WG4qi07PfkWDyfE8M4V,20,0.332,0.932,-2.194,0.0836,0.001380,0.000000,0.1250,0.524,198.816,0.999991
